# Masterclass: Introducción a Hugging Face para NLP

---

## 1. ¿Qué es Hugging Face?

### 1.1 Contextualicemos

Hugging Face es una plataforma que proporciona tres componentes principales:

- **Hub de modelos**

- **Librería transformers**

- **Ecosistema complementario**


### 1.3 Instalación

In [ ]:
# Instalación básica con PyTorch
# !pip install transformers torch

# Instalación con TensorFlow
# !pip install transformers tensorflow

# Instalación completa con dependencias adicionales
# !pip install transformers[torch,sentencepiece,tokenizers]

# Para trabajar con audio
# !pip install transformers[torch,audio]

# Para trabajar con visión
# !pip install transformers[torch,vision]

### Verificación de instalación

In [1]:
import transformers
import torch

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

c:\Users\frasq\Documents\GitHub\Master-IA-y-Big-Data\Entornos-Virtuales\Hugging-Face\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transformers version: 4.57.1
PyTorch version: 2.9.0+cpu
CUDA disponible: False


### 1.4 Configuración de caché

Por defecto, los modelos se descargan en `~/.cache/huggingface/hub`. Para cambiar la ubicación:

In [ ]:
import os

# Establecer directorio de caché personalizado
os.environ['HF_HOME'] = '/ruta/personalizada/cache'

# Verificar configuración
print(f"Directorio de caché: {os.environ.get('HF_HOME', '~/.cache/huggingface')}")

---

## 2. Búsqueda de modelos en Hugging Face

### 2.1 Búsqueda manual en el Hub

Navegación web: https://huggingface.co/models

**Filtros disponibles:**

- **Task**: text-classification, token-classification, question-answering, summarization, translation, text-generation, fill-mask, etc.
- **Library**: transformers, sentence-transformers, spacy, flair
- **Language**: en, es, fr, multilingual, etc.
- **License**: apache-2.0, mit, cc-by-sa-4.0, commercial licenses

### 2.2 Búsqueda programática con Python

In [ ]:
from huggingface_hub import HfApi, list_models

# Crear cliente API
api = HfApi()

# Búsqueda básica: modelos de clasificación de texto
models = list_models(
    filter="text-classification",
    sort="downloads",
    direction=-1,
    limit=10
)

print("Top 10 modelos de clasificación de texto:")
for model in models:
    print(f"- {model.modelId} | Descargas: {model.downloads}")

### 2.3 Búsqueda avanzada con múltiples filtros

In [ ]:
# Buscar modelos de NER en español
models_ner_es = list_models(
    filter={"task": "token-classification", "language": "es"},
    sort="downloads",
    limit=5
)

print("\nModelos de NER en español:")
for model in models_ner_es:
    print(f"- {model.modelId}")

In [ ]:
# Buscar modelos de traducción inglés-español
models_translation = list_models(
    filter="translation",
    search="en-es",
    sort="likes",
    limit=5
)

print("\nModelos de traducción EN-ES:")
for model in models_translation:
    print(f"- {model.modelId} | Likes: {model.likes}")

### 2.4 Categorías principales de tareas

In [ ]:
# Diccionario de tareas disponibles
TASKS = {
    # Texto
    "text-classification": "Clasificación de texto (sentimiento, spam, categorías)",
    "token-classification": "Clasificación de tokens (NER, POS tagging)",
    "question-answering": "Respuesta a preguntas dado un contexto",
    "summarization": "Resumen de textos largos",
    "translation": "Traducción entre idiomas",
    "text-generation": "Generación de texto continuado",
    "fill-mask": "Completar texto con máscaras",
    "text2text-generation": "Transformación texto a texto (paráfrasis, etc.)",
    
    # Audio
    "automatic-speech-recognition": "Transcripción de audio a texto",
    "audio-classification": "Clasificación de audio (género musical, emociones)",
    "text-to-speech": "Síntesis de voz",
    
    # Visión
    "image-classification": "Clasificación de imágenes",
    "object-detection": "Detección de objetos en imágenes",
    "image-segmentation": "Segmentación de imágenes",
    
    # Multimodal
    "visual-question-answering": "Responder preguntas sobre imágenes",
    "image-to-text": "Generar descripciones de imágenes"
}

for task, description in TASKS.items():
    print(f"{task}: {description}")

### 2.5 Recomendaciones de modelos por categoría

In [ ]:
RECOMMENDED_MODELS = {
    # Clasificación de texto - inglés
    "sentiment_en": [
        "distilbert-base-uncased-finetuned-sst-2-english",  # Rápido, eficiente
        "cardiffnlp/twitter-roberta-base-sentiment-latest",  # Especializado en redes sociales
    ],
    
    # Clasificación de texto - español
    "sentiment_es": [
        "finiteautomata/beto-sentiment-analysis",  # BETO adaptado
        "pysentimiento/robertuito-sentiment-analysis",  # Especializado en español latinoamericano
    ],
    
    # NER - inglés
    "ner_en": [
        "dslim/bert-base-NER",  # General purpose
        "dbmdz/bert-large-cased-finetuned-conll03-english",  # Alta precisión
    ],
    
    # NER - español
    "ner_es": [
        "mrm8488/bert-spanish-cased-finetuned-ner",
        "dccuchile/bert-base-spanish-wwm-cased-finetuned-conll02-spanish",
    ],
    
    # Question Answering
    "qa_en": [
        "distilbert-base-cased-distilled-squad",  # Eficiente
        "deepset/roberta-base-squad2",  # Mayor precisión
    ],
    
    # Traducción
    "translation": [
        "Helsinki-NLP/opus-mt-en-es",  # EN -> ES
        "Helsinki-NLP/opus-mt-es-en",  # ES -> EN
    ],
    
    # Summarization
    "summarization": [
        "facebook/bart-large-cnn",  # Noticias
        "t5-base",  # Propósito general
    ],
    
    # Text Generation
    "generation": [
        "gpt2",  # Baseline
        "distilgpt2",  # Más ligero
    ]
}

# Función para obtener recomendaciones
def get_recommended_model(task: str, index: int = 0) -> str:
    """
    Obtiene modelo recomendado para una tarea.
    
    Args:
        task: Clave de la tarea
        index: Índice del modelo (0 = más recomendado)
    
    Returns:
        Nombre del modelo
    """
    if task in RECOMMENDED_MODELS:
        return RECOMMENDED_MODELS[task][index]
    else:
        raise ValueError(f"Tarea {task} no encontrada")

# Ejemplo de uso
model_name = get_recommended_model("sentiment_en", 0)
print(f"Modelo recomendado para sentiment analysis: {model_name}")

### 2.6 Información detallada de un modelo

In [ ]:
from huggingface_hub import model_info

# Obtener metadatos de un modelo
info = model_info("distilbert-base-uncased-finetuned-sst-2-english")

print(f"Modelo: {info.modelId}")
print(f"Tarea: {info.pipeline_tag}")
print(f"Descargas: {info.downloads}")
print(f"Likes: {info.likes}")
print(f"Licencia: {info.cardData.get('license', 'No especificada')}")
print(f"Idiomas: {info.cardData.get('language', 'No especificado')}")

---

## 3. ¿Qué es un Pipeline?

### 3.1 Definición técnica

Un **pipeline** es una abstracción de alto nivel que encapsula tres componentes del proceso de inferencia:

1. **Preprocessing (Tokenizer)**: Convierte texto crudo en tensores numéricos que el modelo puede procesar
2. **Model**: Red neuronal que procesa los tensores y genera representaciones
3. **Postprocessing**: Transforma las salidas del modelo en formato interpretable

### 3.2 Arquitectura interna

#### Proceso manual (sin pipeline)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1. Preprocessing
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
inputs = tokenizer("This is great!", return_tensors="pt")

# 2. Model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
outputs = model(**inputs)

# 3. Postprocessing
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_class = torch.argmax(probs).item()

print(f"Clase predicha: {predicted_class}")
print(f"Probabilidades: {probs}")

#### Mismo proceso usando pipeline

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("This is great!")

print(result)
# [{'label': 'POSITIVE', 'score': 0.9998}]

### 3.3 Ventajas del pipeline

- **Simplificación**: Una línea de código vs. múltiples pasos
- **Configuración automática**: Descarga y configura tokenizer y modelo compatible
- **Manejo de errores**: Gestión automática de casos edge (textos vacíos, muy largos, etc.)
- **Optimizaciones**: Batch processing, device management (CPU/GPU)

### 3.4 Limitaciones del pipeline

- **Menos control**: No acceso a representaciones intermedias
- **Personalización limitada**: Difícil modificar preprocessing o postprocessing
- **Rendimiento**: Overhead en producción de alto volumen

**Recomendación**: Usar pipelines para prototipado rápido y demos. Para producción, considerar implementación manual.

---

## 4. Sintaxis básica para utilizar cualquier modelo

### 4.1 Sintaxis con pipeline (alto nivel)

In [ ]:
from transformers import pipeline

# Sintaxis general
pipeline_obj = pipeline(
    task="nombre-de-tarea",              # Obligatorio
    model="nombre-modelo",                # Opcional (usa modelo default)
    tokenizer="nombre-tokenizer",         # Opcional (usa tokenizer del modelo)
    device=0,                             # Opcional (0=GPU, -1=CPU)
    batch_size=8,                         # Opcional (procesamiento por lotes)
    max_length=512,                       # Opcional (longitud máxima)
    truncation=True                       # Opcional (truncar si excede max_length)
)

# Uso
# resultado = pipeline_obj("texto de entrada")

### 4.2 Sintaxis manual (bajo nivel)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1. Cargar tokenizer y modelo
tokenizer = AutoTokenizer.from_pretrained("nombre-del-modelo")
model = AutoModelForSequenceClassification.from_pretrained("nombre-del-modelo")

# 2. Preparar datos
text = "texto de entrada"
inputs = tokenizer(
    text,
    return_tensors="pt",      # PyTorch tensors
    padding=True,             # Padding a longitud máxima del batch
    truncation=True,          # Truncar si excede max_length
    max_length=512            # Longitud máxima
)

# 3. Inferencia
model.eval()  # Modo evaluación
with torch.no_grad():  # Sin calcular gradientes
    outputs = model(**inputs)

# 4. Procesar salida
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

### 4.3 Clases Auto* para diferentes tareas

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,    # Clasificación de texto
    AutoModelForTokenClassification,       # NER, POS tagging
    AutoModelForQuestionAnswering,         # Question Answering
    AutoModelForSeq2SeqLM,                 # Traducción, summarization
    AutoModelForCausalLM,                  # Generación de texto (GPT)
    AutoModelForMaskedLM                   # Fill-mask (BERT)
)

# Ejemplo: selección automática según arquitectura
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Las clases Auto* detectan automáticamente la arquitectura
model_classification = AutoModelForSequenceClassification.from_pretrained(model_name)
model_masked_lm = AutoModelForMaskedLM.from_pretrained(model_name)

### 4.4 Sintaxis para batch processing

In [ ]:
# Con pipeline
classifier = pipeline("sentiment-analysis")
texts = ["Text 1", "Text 2", "Text 3"]
results = classifier(texts)  # Procesa automáticamente en batch

print(results)

In [ ]:
# Manual
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

texts = ["Text 1", "Text 2", "Text 3"]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

print(predictions)

### 4.5 Gestión de dispositivo (CPU/GPU)

In [4]:
from transformers import pipeline
import torch

# Verificar disponibilidad de CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Con pipeline
classifier = pipeline("sentiment-analysis", device=0)  # 0 = primera GPU

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Usando dispositivo: cpu


c:\Users\frasq\Documents\GitHub\Master-IA-y-Big-Data\Entornos-Virtuales\Hugging-Face\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\frasq\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Manual
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model.to(device)

inputs = tokenizer("texto", return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}  # Mover inputs a GPU

outputs = model(**inputs)

---

## 5. Ejemplos prácticos

### 5.1 Ejemplo 1: Análisis de sentimiento

In [ ]:
from transformers import pipeline

# Cargar pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# Caso de uso: Analizar reviews de productos
reviews = [
    "This product exceeded all my expectations. Highly recommended!",
    "Terrible quality. Broke after one day of use.",
    "It's okay, nothing special but does the job.",
    "Best purchase I've made this year!"
]

results = sentiment_analyzer(reviews)

# Mostrar resultados
for review, result in zip(reviews, results):
    print(f"Review: {review}")
    print(f"Sentiment: {result['label']} (confidence: {result['score']:.4f})")
    print("-" * 80)

**Aplicación real**: Amazon procesa millones de reviews diariamente para calcular ratings ajustados por sentimiento.

### 5.2 Ejemplo 2: Named Entity Recognition

In [ ]:
from transformers import pipeline

# Cargar pipeline de NER
ner = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    grouped_entities=True  # Agrupa tokens de la misma entidad
)

# Caso de uso: Extraer información de artículos de noticias
text = """
Apple Inc. announced that CEO Tim Cook will visit the new headquarters 
in Cupertino, California next Monday. The company reported revenue of 
$394 billion in 2023.
"""

entities = ner(text)

# Mostrar entidades encontradas
print("Entidades detectadas:")
for entity in entities:
    print(f"- {entity['word']}: {entity['entity_group']} (score: {entity['score']:.4f})")

**Aplicación real**: Reuters utiliza NER para etiquetar automáticamente noticias y crear bases de datos de eventos.

### 5.3 Ejemplo 3: Question Answering

In [ ]:
from transformers import pipeline

# Cargar pipeline
qa_model = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad"
)

# Caso de uso: Sistema de FAQ automatizado
context = """
Hugging Face is a company that develops tools for building applications 
using machine learning. The company was founded in 2016 and is based in 
New York City. Their main product is the Transformers library, which 
provides APIs and tools to download and train state-of-the-art pretrained 
models. The library supports PyTorch, TensorFlow, and JAX.
"""

questions = [
    "When was Hugging Face founded?",
    "Where is the company based?",
    "What frameworks does the library support?"
]

# Responder preguntas
for question in questions:
    result = qa_model(question=question, context=context)
    print(f"Q: {question}")
    print(f"A: {result['answer']} (score: {result['score']:.4f})")
    print()

**Aplicación real**: Zendesk implementa QA para responder automáticamente tickets de soporte basándose en documentación.

### 5.4 Ejemplo 4: Traducción automática

In [ ]:
from transformers import pipeline

# Cargar pipeline de traducción
translator_en_es = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-en-es"
)

# Caso de uso: Traducir documentación técnica
english_texts = [
    "Machine learning models require large amounts of data for training.",
    "The transformer architecture revolutionized natural language processing.",
    "Python is the most popular programming language for data science."
]

# Traducir
translations = translator_en_es(english_texts)

# Mostrar resultados
for original, translation in zip(english_texts, translations):
    print(f"EN: {original}")
    print(f"ES: {translation['translation_text']}")
    print()

**Aplicación real**: Microsoft Translator API utiliza arquitecturas similares para traducción en tiempo real en Teams y Skype.

### 5.5 Ejemplo 5: Fill-mask (completar texto)

In [ ]:
from transformers import pipeline

# Cargar pipeline
unmasker = pipeline(
    "fill-mask",
    model="bert-base-uncased"
)

# Caso de uso: Sugerencias de autocompletado
text_with_mask = "The capital of France is [MASK]."

# Obtener predicciones
predictions = unmasker(text_with_mask)

# Mostrar top 5 predicciones
print(f"Texto: {text_with_mask}")
print("\nPredicciones:")
for i, pred in enumerate(predictions, 1):
    print(f"{i}. {pred['token_str']}: {pred['score']:.4f}")

**Aplicación real**: Gmail utiliza modelos similares para Smart Compose (sugerencias de autocompletado).

### 5.6 Ejemplo 6: Implementación manual con control detallado

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

class SentimentClassifier:
    """
    Clasificador de sentimiento con control manual del proceso.
    """
    
    def __init__(self, model_name: str, device: str = "cpu"):
        """
        Inicializa el clasificador.
        
        Args:
            model_name: Nombre del modelo en Hugging Face
            device: 'cpu' o 'cuda'
        """
        self.device = torch.device(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        
        # Mapeo de etiquetas
        self.labels = {0: "NEGATIVE", 1: "POSITIVE"}
    
    def predict(self, text: str, return_probs: bool = False) -> dict:
        """
        Predice el sentimiento de un texto.
        
        Args:
            text: Texto a clasificar
            return_probs: Si True, devuelve probabilidades de todas las clases
            
        Returns:
            dict con label, score y opcionalmente probabilities
        """
        # Tokenización
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        )
        
        # Mover a dispositivo
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Inferencia
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)
        
        # Predicción
        predicted_class = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][predicted_class].item()
        
        result = {
            "label": self.labels[predicted_class],
            "score": confidence
        }
        
        if return_probs:
            result["probabilities"] = {
                label: probs[0][idx].item() 
                for idx, label in self.labels.items()
            }
        
        return result
    
    def predict_batch(self, texts: list) -> list:
        """
        Predice el sentimiento de múltiples textos.
        
        Args:
            texts: Lista de textos
            
        Returns:
            Lista de diccionarios con predicciones
        """
        # Tokenización por lotes
        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        )
        
        # Mover a dispositivo
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Inferencia
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)
        
        # Procesar resultados
        results = []
        for i in range(len(texts)):
            predicted_class = torch.argmax(probs[i]).item()
            confidence = probs[i][predicted_class].item()
            
            results.append({
                "label": self.labels[predicted_class],
                "score": confidence
            })
        
        return results

In [ ]:
# Uso del clasificador

# Inicializar
classifier = SentimentClassifier(
    model_name="distilbert-base-uncased-finetuned-sst-2-english",
    device="cpu"
)

# Predicción individual
result = classifier.predict(
    "This product is amazing!",
    return_probs=True
)
print("Predicción individual:")
print(result)
print()

# Predicción por lotes
texts = [
    "Great service and fast delivery!",
    "Disappointed with the quality.",
    "Average product, nothing special."
]

results = classifier.predict_batch(texts)
print("Predicción por lotes:")
for text, result in zip(texts, results):
    print(f"{text} -> {result['label']} ({result['score']:.4f})")

### 5.7 Ejemplo 7: Manejo de errores y casos edge

In [ ]:
from transformers import pipeline
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_sentiment_analysis(texts: list) -> list:
    """
    Análisis de sentimiento con manejo de errores.
    
    Args:
        texts: Lista de textos a analizar
        
    Returns:
        Lista de resultados con manejo de errores
    """
    try:
        classifier = pipeline("sentiment-analysis")
        results = []
        
        for text in texts:
            try:
                # Validar entrada
                if not isinstance(text, str):
                    logger.warning(f"Input no es string: {type(text)}")
                    results.append({"error": "Invalid input type"})
                    continue
                
                if len(text.strip()) == 0:
                    logger.warning("Texto vacío")
                    results.append({"error": "Empty text"})
                    continue
                
                # Truncar si es muy largo
                if len(text) > 10000:
                    logger.warning(f"Texto muy largo ({len(text)} caracteres), truncando")
                    text = text[:10000]
                
                # Predecir
                result = classifier(text)[0]
                results.append(result)
                
            except Exception as e:
                logger.error(f"Error procesando texto: {str(e)}")
                results.append({"error": str(e)})
        
        return results
    
    except Exception as e:
        logger.error(f"Error inicializando pipeline: {str(e)}")
        raise

# Prueba con casos edge
test_cases = [
    "Normal text",
    "",  # Texto vacío
    None,  # No string
    "A" * 15000,  # Texto muy largo
    "Special chars: @#$%^&*()"
]

results = safe_sentiment_analysis(test_cases)
for case, result in zip(test_cases, results):
    case_preview = str(case)[:50] if case else str(case)
    print(f"Input: {case_preview}")
    print(f"Result: {result}")
    print()

---

## Ejercicios propuestos

1. **Ejercicio básico**: Implementar un clasificador de spam que procese correos electrónicos y determine si son spam o legítimos.

2. **Ejercicio intermedio**: Crear un sistema que extraiga nombres de personas, organizaciones y ubicaciones de artículos de noticias y los almacene en un diccionario estructurado.

3. **Ejercicio avanzado**: Desarrollar un asistente de FAQ que tome documentación técnica como contexto y responda preguntas de usuarios, con logging de confianza de las respuestas.

---

## Recursos adicionales

- Documentación oficial: https://huggingface.co/docs/transformers
- Model Hub: https://huggingface.co/models
- Datasets Hub: https://huggingface.co/datasets
- Course: https://huggingface.co/course
- Papers with Code: https://paperswithcode.com (para entender arquitecturas base)